# IMPORTS

In [ ]:
import os
from classes.layers.Conv2D import Conv2D
from classes.layers.Dense import Dense
from classes.layers.Flatten import Flatten
from classes.layers.Input import InputLayer
from classes.models.Sequential import Sequential
from classes.utils.ImageConvert import ImageConvert
from classes.losses.BinaryCrossentropy import BinaryCrossentropy

import numpy as np

# DATA PREPROCESSING

In [ ]:
ic_train = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_train_gen = ic_train.from_directory(os.path.join('.','data','train'), (256,256), mode='binary', color_mode='rgb')

ic_test = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_test_gen = ic_test.from_directory(os.path.join('.','data','test'), (256,256), mode='binary', color_mode='rgb')

In [ ]:
X_train = []
y_train = []

for i in range(len(data_train_gen)):
    step = next(data_train_gen)
    X_train.append(step['data'])
    y_train.append(step['label'])

# MODEL

## Instantiating

In [ ]:
def generate_n_model(n):
    models = []
    for _ in range(n):
        model = Sequential([
                InputLayer(input_shape=(256,256,3)),
                Conv2D(2, (16, 16), activation='relu'),
                Flatten(),
                Dense(16, activation='relu'),
                Dense(1, activation='sigmoid')
            ],  name=f'Model_{_}')

        model.compile(loss=BinaryCrossentropy())
        models.append(model)
    return models

In [ ]:
models = generate_n_model(10)

## Training

In [ ]:
def cross_validation_split(data, label, n):
    data_split = [list(_) for _ in np.array_split(data, n)]
    label_split = [list(_) for _ in np.array_split(label, n)]
    
    train_split = []
    valid_split = []
    label_train_split = []
    label_valid_split = []

    for i in range(len(data_split)):
        # Separate validation k fold split
        for data in data_split[i]:
            valid_split.append(data)
        for label in label_split[i]:
            label_valid_split.append(label)
        # Union train k fold split
        temp_train_data = []
        for data in data_split[:i]:
            # for data in set:
            temp_train_data += list(data)
        for data in data_split[i+1:]:
            # for data in set:
            temp_train_data += list(data)
        train_split.append(temp_train_data)
        
        temp_train_label = []
        for set in label_split[:i]:
            for label in set:
                temp_train_label.append(label)
        for set in label_split[i+1:]:
            for label in set:
                temp_train_label.append(label)
        label_train_split.append(temp_train_label)
    
    return train_split, valid_split, label_train_split, label_valid_split

train_split, valid_split,\
    label_train_split, label_valid_split = cross_validation_split(X_train, y_train, 10)

In [ ]:
for i in range(len(models)):
    print(models[i].name)
    models[i].fit(train_split[i], label_train_split[i], 2, 1, 0.2)
    print()
    print()

## Testing

In [ ]:
# TEST
for model in models:
    print(model.name)
    model.predict(data_test_gen)

## Pick Best Model

In [ ]:
model = models[2]

# SAVE & LOAD

In [ ]:
model.save("save_model")

In [ ]:
model.load("save_model.json")

In [ ]:
model.predict(X_test)